In [2]:
import keras
from keras.models import Sequential
from keras import layers
from keras.layers import Dense,InputLayer, Dropout
from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score
import pandas as pd
import datetime

import numpy as np
import tensorflow as tf
from scikeras.wrappers import KerasClassifier
import keras_tuner
from keras.metrics import categorical_accuracy
from tensorflow.keras import regularizers
from sklearn import preprocessing
import multiprocessing


# Global Model

In [3]:
class MyHyperModel(keras_tuner.HyperModel):
    def build(self,hp):
        model = Sequential()
        model.add(InputLayer(input_shape=(240, )))
        hp_units = hp.Choice('units', values=[32,64,128])
        model.add(Dense(units=hp_units, name='dense_0'))
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='sigmoid'))
        hp_lr = hp.Choice('learning_rate', values=[1e-6, 1e-7, 1e-8])
        hp_optimizer = hp.Choice('optimizer', values=['sgd', 'rmsprop', 'adam',"adamax"])
        if hp_optimizer == 'sgd':
            optimizer = keras.optimizers.SGD(learning_rate=hp_lr)
        elif hp_optimizer == 'rmsprop':
            optimizer = keras.optimizers.RMSprop(learning_rate=hp_lr)
        elif hp_optimizer == 'adam':
            optimizer = keras.optimizers.Adam(learning_rate=hp_lr)
        elif hp_optimizer == 'adamax':
            optimizer = keras.optimizers.Adamax(learning_rate=hp_lr)
        else:
            raise ValueError("Invalid optimizer choice")

        model.compile(
            optimizer = optimizer
            ,loss=keras.losses.BinaryCrossentropy()
            , metrics=['accuracy'])
        return model


    def fit(self, hp, model, *args, **kwargs):
            return model.fit(
                *args,
                batch_size=hp.Choice("batch_size", [16,32, 64,128,256]),
                **kwargs,
            )


In [4]:
timesteps = 240
num_input =1
num_classes=1
label = list(range(timesteps)) + ['target'] + ['ticker'] + ['target_date'] + ['sector']

training_data = []
training_label = []
testing_data =[]
testing_label =[]

accuracy_results = []

for i in range(5):
    # read the data
    data_path = '/SP500/data/'
    train = pd.read_csv(data_path+'Set_' + str(i) + '_Train.csv', index_col=0).dropna()
    test = pd.read_csv(data_path+'Set_' + str(i) + '_Test.csv', index_col=0).dropna()

    train.columns = label
    test.columns = label

    train_label = train.iloc[:, timesteps]
    train_data = train.iloc[:, :timesteps]
    test_label = test.iloc[:,timesteps]
    test_data = test.iloc[:, :timesteps]
    
    
     # reshape input
    #  data: (samples, timesteps, features)
    x_train = np.array(train_data).reshape((len(train_data), timesteps, num_input), order = 'F')
    x_test = np.array(test_data).reshape((len(test_data), timesteps, num_input), order = 'F')
    # label: (samples, target)
    y_train = np.array(train_label).reshape((len(train_label), num_classes))
    y_test = np.array(test_label).reshape((len(test_label), num_classes))
        
    print(x_train.shape)
    print(y_train.shape)
    print(x_test.shape)
    print(y_test.shape)
    
    print("-------------------------------------------------------------------------------------------------------")
    print("Training the model for Training Set " + str(i) + " from " +
    datetime.datetime.strftime(datetime.datetime.now(), '%Y-%m-%d %H:%M:%S'))
    print("-------------------------------------------------------------------------------------------------------")
        
    if i==0:    
        tuner = keras_tuner.BayesianOptimization(MyHyperModel(),
                objective='val_accuracy', # overwrite=True,
                max_trials=30, directory='fnn', seed=77)
        early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 3, restore_best_weights=False)
        tuner.search(x_train,y_train, epochs =1000,validation_split=0.2,callbacks=[early_stop])

        # save the best model
        hypermodel = MyHyperModel()
        best_hp = tuner.get_best_hyperparameters()[0]
        print(tuner.get_best_hyperparameters()[0].get_config()["values"])
        best_model = hypermodel.build(best_hp)
        early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 10, restore_best_weights=False)
        result = best_model.fit(x_train,y_train, epochs=1000, validation_split =0.2, verbose =1, callbacks=[early_stop])


    else:
        load_path="/SP500/4_FNN/fnn_model_weight/fnn_weight1.h5"
        print('Model restore from '+load_path)
        cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=load_path,
                                                     save_weights_only=True,
                                                     verbose=1)
        early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                        patience = 10, restore_best_weights=False)
        
        
        result = best_model.fit(
                x_train, 
                y_train, 
                epochs = 1000, 
                validation_split=0.2,
                verbose =1,
                callbacks=[cp_callback, early_stop]        
            )         
      
        
    save_path = "/SP500/4_FNN/fnn_model_weight/fnn_weight1.h5"
    best_model.save_weights(save_path)
    print("Model saved to " + save_path)
    print("Training end: " + datetime.datetime.strftime(datetime.datetime.now(), '%Y-%m-%d %H:%M:%S'))
        
    ##make prediction
    pred_ff_test = best_model.predict(x_test)
    #pred = pred_ff_test.tolist()
    pred = pred_ff_test.reshape((1, len(pred_ff_test))).tolist()[0]
    output_data = pd.DataFrame({'y_prob': pred, 'y_true': test['target'], 'Ticker': test['ticker'],
                                    'Date': test['target_date'], 'Sector': test['sector'], })
    accuracy = accuracy_score(np.round(output_data['y_prob']), output_data['y_true'])
    print('Overall Accuracy for test set:'+ str(accuracy))
    output_data.to_csv('/SP500/4_FNN/fnn_pred/fnn_prediction_period_'+str(i)+'.csv')
    print('Prediction for period ' + str(i) + ' successfully saved.')

Trial 30 Complete [00h 31m 23s]
val_accuracy: 0.5162243247032166

Best val_accuracy So Far: 0.5175687074661255
Total elapsed time: 13h 19m 04s
{'units': 64, 'learning_rate': 1e-06, 'optimizer': 'adam', 'batch_size': 32}
Epoch 1/1000
6137/6137 [==============================] - 21s 3ms/step - loss: 1.2521 - accuracy: 0.4990 - val_loss: 1.0101 - val_accuracy: 0.4993
Epoch 2/1000
6137/6137 [==============================] - 20s 3ms/step - loss: 1.2178 - accuracy: 0.4974 - val_loss: 0.9774 - val_accuracy: 0.4992
Epoch 3/1000
6137/6137 [==============================] - 19s 3ms/step - loss: 1.1880 - accuracy: 0.4985 - val_loss: 0.9477 - val_accuracy: 0.4996
Epoch 4/1000
6137/6137 [==============================] - 19s 3ms/step - loss: 1.1553 - accuracy: 0.5014 - val_loss: 0.9207 - val_accuracy: 0.4998
Epoch 5/1000
6137/6137 [==============================] - 19s 3ms/step - loss: 1.1344 - accuracy: 0.4989 - val_loss: 0.8963 - val_accuracy: 0.5004
Epoch 6/1000
6137/6137 [=====================

Epoch 55/1000
6137/6137 [==============================] - 18s 3ms/step - loss: 0.8089 - accuracy: 0.4995 - val_loss: 0.6935 - val_accuracy: 0.5085
Epoch 56/1000
6137/6137 [==============================] - 19s 3ms/step - loss: 0.8045 - accuracy: 0.5034 - val_loss: 0.6934 - val_accuracy: 0.5091
Epoch 57/1000
6137/6137 [==============================] - 18s 3ms/step - loss: 0.8017 - accuracy: 0.5029 - val_loss: 0.6934 - val_accuracy: 0.5093
Epoch 58/1000
6137/6137 [==============================] - 19s 3ms/step - loss: 0.7999 - accuracy: 0.5031 - val_loss: 0.6933 - val_accuracy: 0.5086
Epoch 59/1000
6137/6137 [==============================] - 19s 3ms/step - loss: 0.7960 - accuracy: 0.5039 - val_loss: 0.6932 - val_accuracy: 0.5098
Epoch 60/1000
6137/6137 [==============================] - 19s 3ms/step - loss: 0.7951 - accuracy: 0.5038 - val_loss: 0.6932 - val_accuracy: 0.5098
Epoch 61/1000
6137/6137 [==============================] - 19s 3ms/step - loss: 0.7944 - accuracy: 0.5021 - val_

6137/6137 [==============================] - 18s 3ms/step - loss: 0.7006 - accuracy: 0.5066 - val_loss: 0.6922 - val_accuracy: 0.5156
Epoch 166/1000
6137/6137 [==============================] - 18s 3ms/step - loss: 0.7004 - accuracy: 0.5049 - val_loss: 0.6922 - val_accuracy: 0.5156
Epoch 167/1000
6137/6137 [==============================] - 20s 3ms/step - loss: 0.7000 - accuracy: 0.5057 - val_loss: 0.6922 - val_accuracy: 0.5157
Epoch 168/1000
6137/6137 [==============================] - 20s 3ms/step - loss: 0.6990 - accuracy: 0.5079 - val_loss: 0.6922 - val_accuracy: 0.5155
Epoch 169/1000
6137/6137 [==============================] - 20s 3ms/step - loss: 0.6990 - accuracy: 0.5081 - val_loss: 0.6922 - val_accuracy: 0.5157
Epoch 170/1000
6137/6137 [==============================] - 18s 3ms/step - loss: 0.6987 - accuracy: 0.5085 - val_loss: 0.6922 - val_accuracy: 0.5159
Epoch 171/1000
6137/6137 [==============================] - 21s 3ms/step - loss: 0.6989 - accuracy: 0.5067 - val_loss: 0.

Epoch 9/1000
6126/6130 [============================>.] - ETA: 0s - loss: 0.6955 - accuracy: 0.5035
Epoch 9: saving model to /home/RDC/yeungwin/H:/yeungwin/SP500/4_FNN/fnn_model_weight/fnn_weight1.h5
6130/6130 [==============================] - 18s 3ms/step - loss: 0.6955 - accuracy: 0.5035 - val_loss: 0.6928 - val_accuracy: 0.5058
Epoch 10/1000
6120/6130 [============================>.] - ETA: 0s - loss: 0.6953 - accuracy: 0.5050
Epoch 10: saving model to /home/RDC/yeungwin/H:/yeungwin/SP500/4_FNN/fnn_model_weight/fnn_weight1.h5
6130/6130 [==============================] - 21s 3ms/step - loss: 0.6953 - accuracy: 0.5049 - val_loss: 0.6928 - val_accuracy: 0.5056
Epoch 11/1000
6124/6130 [============================>.] - ETA: 0s - loss: 0.6952 - accuracy: 0.5055
Epoch 11: saving model to /home/RDC/yeungwin/H:/yeungwin/SP500/4_FNN/fnn_model_weight/fnn_weight1.h5
6130/6130 [==============================] - 18s 3ms/step - loss: 0.6952 - accuracy: 0.5055 - val_loss: 0.6928 - val_accuracy: 0

6130/6130 [==============================] - 18s 3ms/step - loss: 0.6939 - accuracy: 0.5069 - val_loss: 0.6928 - val_accuracy: 0.5042
Epoch 34/1000
6126/6130 [============================>.] - ETA: 0s - loss: 0.6937 - accuracy: 0.5063
Epoch 34: saving model to /home/RDC/yeungwin/H:/yeungwin/SP500/4_FNN/fnn_model_weight/fnn_weight1.h5
6130/6130 [==============================] - 19s 3ms/step - loss: 0.6937 - accuracy: 0.5063 - val_loss: 0.6928 - val_accuracy: 0.5044
Epoch 35/1000
6128/6130 [============================>.] - ETA: 0s - loss: 0.6933 - accuracy: 0.5099
Epoch 35: saving model to /home/RDC/yeungwin/H:/yeungwin/SP500/4_FNN/fnn_model_weight/fnn_weight1.h5
6130/6130 [==============================] - 18s 3ms/step - loss: 0.6933 - accuracy: 0.5099 - val_loss: 0.6928 - val_accuracy: 0.5044
Epoch 36/1000
6108/6130 [============================>.] - ETA: 0s - loss: 0.6934 - accuracy: 0.5083
Epoch 36: saving model to /home/RDC/yeungwin/H:/yeungwin/SP500/4_FNN/fnn_model_weight/fnn_wei

...

3727/3727 [==============================] - 7s 2ms/step
Overall Accuracy for test set:0.5078620967268519
Prediction for period 3 successfully saved.
(244381, 240, 1)
(244381, 1)
(119325, 240, 1)
(119325, 1)
-------------------------------------------------------------------------------------------------------
Training the model for Training Set 4 from 2024-03-12 05:57:22
-------------------------------------------------------------------------------------------------------
Model restore from /home/RDC/yeungwin/H:/yeungwin/SP500/4_FNN/fnn_model_weight/fnn_weight1.h5
Epoch 1/1000
6100/6110 [============================>.] - ETA: 0s - loss: 0.6929 - accuracy: 0.5139
Epoch 1: saving model to /home/RDC/yeungwin/H:/yeungwin/SP500/4_FNN/fnn_model_weight/fnn_weight1.h5
6110/6110 [==============================] - 19s 3ms/step - loss: 0.6929 - accuracy: 0.5140 - val_loss: 0.6931 - val_accuracy: 0.5105
Epoch 2/1000
6099/6110 [============================>.] - ETA: 0s - loss: 0.6928 - accuracy: 

Epoch 24/1000
6103/6110 [============================>.] - ETA: 0s - loss: 0.6921 - accuracy: 0.5179
Epoch 24: saving model to /home/RDC/yeungwin/H:/yeungwin/SP500/4_FNN/fnn_model_weight/fnn_weight1.h5
6110/6110 [==============================] - 18s 3ms/step - loss: 0.6921 - accuracy: 0.5179 - val_loss: 0.6926 - val_accuracy: 0.5123
Epoch 25/1000
6090/6110 [============================>.] - ETA: 0s - loss: 0.6922 - accuracy: 0.5179
Epoch 25: saving model to /home/RDC/yeungwin/H:/yeungwin/SP500/4_FNN/fnn_model_weight/fnn_weight1.h5
6110/6110 [==============================] - 18s 3ms/step - loss: 0.6922 - accuracy: 0.5179 - val_loss: 0.6926 - val_accuracy: 0.5121
Epoch 26/1000
6105/6110 [============================>.] - ETA: 0s - loss: 0.6921 - accuracy: 0.5179
Epoch 26: saving model to /home/RDC/yeungwin/H:/yeungwin/SP500/4_FNN/fnn_model_weight/fnn_weight1.h5
6110/6110 [==============================] - 18s 3ms/step - loss: 0.6921 - accuracy: 0.5179 - val_loss: 0.6926 - val_accuracy:

6110/6110 [==============================] - 18s 3ms/step - loss: 0.6919 - accuracy: 0.5183 - val_loss: 0.6925 - val_accuracy: 0.5125
Epoch 49/1000
6104/6110 [============================>.] - ETA: 0s - loss: 0.6918 - accuracy: 0.5182
Epoch 49: saving model to /home/RDC/yeungwin/H:/yeungwin/SP500/4_FNN/fnn_model_weight/fnn_weight1.h5
6110/6110 [==============================] - 19s 3ms/step - loss: 0.6918 - accuracy: 0.5182 - val_loss: 0.6925 - val_accuracy: 0.5124
Epoch 50/1000
6097/6110 [============================>.] - ETA: 0s - loss: 0.6918 - accuracy: 0.5174
Epoch 50: saving model to /home/RDC/yeungwin/H:/yeungwin/SP500/4_FNN/fnn_model_weight/fnn_weight1.h5
6110/6110 [==============================] - 21s 4ms/step - loss: 0.6918 - accuracy: 0.5174 - val_loss: 0.6925 - val_accuracy: 0.5127
Epoch 51/1000
6098/6110 [============================>.] - ETA: 0s - loss: 0.6918 - accuracy: 0.5196
Epoch 51: saving model to /home/RDC/yeungwin/H:/yeungwin/SP500/4_FNN/fnn_model_weight/fnn_wei

Epoch 73/1000
6096/6110 [============================>.] - ETA: 0s - loss: 0.6916 - accuracy: 0.5191
Epoch 73: saving model to /home/RDC/yeungwin/H:/yeungwin/SP500/4_FNN/fnn_model_weight/fnn_weight1.h5
6110/6110 [==============================] - 19s 3ms/step - loss: 0.6916 - accuracy: 0.5190 - val_loss: 0.6924 - val_accuracy: 0.5127
Epoch 74/1000
6110/6110 [==============================] - ETA: 0s - loss: 0.6916 - accuracy: 0.5185
Epoch 74: saving model to /home/RDC/yeungwin/H:/yeungwin/SP500/4_FNN/fnn_model_weight/fnn_weight1.h5
6110/6110 [==============================] - 19s 3ms/step - loss: 0.6916 - accuracy: 0.5185 - val_loss: 0.6924 - val_accuracy: 0.5126
Epoch 75/1000
6102/6110 [============================>.] - ETA: 0s - loss: 0.6915 - accuracy: 0.5202
Epoch 75: saving model to /home/RDC/yeungwin/H:/yeungwin/SP500/4_FNN/fnn_model_weight/fnn_weight1.h5
6110/6110 [==============================] - 19s 3ms/step - loss: 0.6915 - accuracy: 0.5202 - val_loss: 0.6924 - val_accuracy:

6110/6110 [==============================] - 19s 3ms/step - loss: 0.6915 - accuracy: 0.5198 - val_loss: 0.6924 - val_accuracy: 0.5121
Epoch 98/1000
6089/6110 [============================>.] - ETA: 0s - loss: 0.6915 - accuracy: 0.5192
Epoch 98: saving model to /home/RDC/yeungwin/H:/yeungwin/SP500/4_FNN/fnn_model_weight/fnn_weight1.h5
6110/6110 [==============================] - 19s 3ms/step - loss: 0.6915 - accuracy: 0.5192 - val_loss: 0.6924 - val_accuracy: 0.5120
Epoch 99/1000
6087/6110 [============================>.] - ETA: 0s - loss: 0.6913 - accuracy: 0.5213
Epoch 99: saving model to /home/RDC/yeungwin/H:/yeungwin/SP500/4_FNN/fnn_model_weight/fnn_weight1.h5
6110/6110 [==============================] - 21s 3ms/step - loss: 0.6913 - accuracy: 0.5212 - val_loss: 0.6924 - val_accuracy: 0.5126
Epoch 100/1000
6093/6110 [============================>.] - ETA: 0s - loss: 0.6914 - accuracy: 0.5200
Epoch 100: saving model to /home/RDC/yeungwin/H:/yeungwin/SP500/4_FNN/fnn_model_weight/fnn_w